# Jrfapp tutorial 4 
## Inverting real dataset for station CAD_P with PSO:

First we need to import Initialize and Jrfapp_station. You can perform it by:

In [1]:
from jrfapp.main_classes import Initialize
from jrfapp.main_classes import Jrfapp_station
import os

In this run we are going to use real dataset of CAD_P station in the joint inversion framework. The sac file provided
in the station folder should have T0, Omarker, the USER4 must be ray parameter. We first need to create a Initialize object and refrence the data_folder and stations coordinate file. Similar to toturial 2, we can do it by:

In [2]:
### Folders that need to be defined:

## Path to the data folder. This folder must contain your stations folder data. 
data_folder = '/mnt/4TB/soroush/CB_network'
###############################################################################
## Path to the folder which contains references models. The folder must only 
## contain .dat files. 
model_folder = '/home/soroush/my_py_rf/model_folder'
###############################################################################
## Path to the specific refrence initial model. These option used because you may want to
## use different model as the reference model. 
model_name='/home/soroush/my_py_rf/model_folder/continent_tibet_model.dat'
###############################################################################
## Path to the coordinates file. This file must contain station name, lat and 
## long of the stations. Note that the station name in this file, the station 
## name in the data_folder and station name given to thr Jrfapp_station class 
## must be same.
station_coordinate_file= '/home/soroush/my_py_rf/CB_coordinates'
###############################################################################
## The folder which package use for saving the output of stations. 
output_folder='/mnt/4TB/soroush/CB_st/syn_st'

In [3]:
init_obj = Initialize(network_name='CB', 
                                    station_coordinate_file= station_coordinate_file,
                                    data_folder= data_folder, 
                                    layering= [4, 5, 5], 
                                    model_folder = model_folder,
                                    output_folder= output_folder, 
                                    model_name= model_name,
                                    random_seed= 250)

The CB_coordinates file contains the station name, latitude and longitude. 
for example 
> CAD_P     31.0    97.50

is one of the lines in CB_coordinates that correspond to CAD_P station.

The data_folder is where we saved the dataset of CAD_P station. the data_folder structures must be as follow:
> {data_folder}/{station_folder}/{event_folder}/*BHZ.SAC *BHE.SAC *BHN.SAC
>for example:
>> data_folder/CAD_P/2013031033343/CAD_2013031033343_BHE.SAC,

>> data_folder/CAD_P/2013031033343/CAD_2013031033343_BHN.SAC,  

>> data_folder/CAD_P/2013031033343/CAD_2013031033343_BHZ.SAC.

Now we need to create Jrfapp_station with CAD_P dataset. This can be done as follow:

In [ ]:
jrfapp_stobj = Jrfapp_station(init_obj, name = 'CAD_P')

Creating station CAD_P 
found 2143 events from which 
 829 had 3 waveforms
reading files for network = CB ,station = CAD_P
all data saved for network= CB ,station= CAD_P to:
/mnt/4TB/soroush/CB_st/syn_st/CB_CAD_P/CB_CAD_P_all_data_bf_rf.bin
preparing files for rf calculation for network = CB, station = CAD_P


/home/soroush/.conda/envs/rfenv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/soroush/.conda/envs/rfenv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/soroush/.conda/envs/rfenv/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/soroush/.conda/envs/rfenv/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/soroush/.conda/envs/rfenv/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/soroush/.conda/envs/rfenv/lib/python3.11/site-pac

The backbone of Jrfapp_station is the Station_sor class. This class read the dataset, decimates and filters the data according to init_obj, and calculates RFs and Apparent velocities. Remove low-quality dataset, create several figures in output_folder/{station_name} folder, and prepare several stacks. 
These stacks are:
#### 1. "Linear Stack before app_vel criteria":  
> which is stacked RFs before
> removing RFs that produce apparent velocities lower than 
> min_app_vel and greater than max_app_vel defined in the Initialize 
> class. I don't recommend using this because unrealistic values for 
> Apparent velocities are possibly generated by noise in the dataset.            
#### 2. "Linear Stack after app_vel criteria", 
> which is stacked RFs before
> removing RFs that produce apparent velocities lower than 
> min_app_vel and greater than max_app_vel defined in the Initialize 
> class. This is the recommended parameter for inversion. 
            
#### 3. "Phase Weighted stack after app_vel criteria": 
> which is the phase 
> weighted stack of RFs after removing RFs which produce apparent
> velocities lower than min_app_vel and greater than max_app_vel
> defined in the Initialize class. I don't recommend using this
> because moveout correction didn't apply to RFs in the phase 
> weighted stacking process and most of the multiple is removed 
> from RF time series.
            
#### 4. "K0 Stack joint_harmonic before app_vel criteria": 
> K0 stack is 
> based on removing first-order variation of azimuthal difference in 
> RFs. The basic idea is to keep the bulk value of each phase in the 
> RFR time series. However, this stacking method needs a reliable dataset
> to constrain the azimuthal variation. This stack is the K0 stack before
> removing RFs that produce apparent velocities lower than 
> min_app_vel and greater than max_app_vel defined in the Initialize 
> class. I don't recommend using this unless you have enough back azimuth
> variation in a low-noise level dataset.
            
#### 5. "K0 Stack joint_harmonic after app_vel criteria": 
> like previous but
> After removing RFs that produce apparent velocities lower than 
> min_app_vel and greater than max_app_vel defined in the Initialize 
> class. You can use this when you have a very good back azimuth 
> variation.
           
#### 6. "Weighted Stack according to Number of trace in each bin before app_vel criteria":
> linear stack but weighted according to the
> number of traces in each bin of back azimuth. In the process of 
> creating the Station the back azimuth is divided into 24 bins and RFs in 
> each bin are stacked. This stacking method uses the number of RFs in 
> each stack as the weight for stacking all 24 bins. This 
> stack calculated before removing RFs that produce apparent velocities
> lower than min_app_vel and greater than max_app_vel defined in the
> Initialize class. I don't recommend using this cause of noises that
> produce unrealistic apparent velocity.
            
#### 8. "Weighted Stack according to Number of trace in each bin after app_vel criteria":
> same as 6 but after removing RFs which 
> produce apparent velocities lower than min_app_vel and greater 
> than max_app_vel defined in the Initialize class. You can use this
> when your RFs dominated by a specific back azimuth. 
            
#### 10. "Linear Stack of bins before app_vel criteria": 
> same as 1 but 
            each bins have a weight equal to 1.

#### 11. "Linear Stack of bins after app_vel criteria": 
> same as 8 but after 
> removing RFs that produce apparent velocities lower than min_app_vel 
> and greater than max_app_vel defined in the Initialize class.
      
#### 12. "synthetic":
> for inverting synthetic data made from real datasets.

<div class="alert alert-block alert-info">
<b>Tip:</b> It is good practice to save the jrfapp_stobj before performing inversion. you can do it as follow:
</div>

In [9]:
jrfapp_stobj_file_name = jrfapp_stobj.save_file(file_name='real_data_cad_bf_inv.bin')

Saved to /home/soroush/rf_shallow_codes/makran_data/pkg_test_real_chbr_gs/real_data_chbr_gs


Now we invert the "K0 Stack joint_harmonic after app_vel criteria" which is K0 stack for CAD_P station. This
can be done with either grid_search or PSO. For inversion using PSO, we have:

In [ ]:
jrfapp_stobj.invert_data(inv_method = 'PSO',
                stack_name = "K0 Stack joint_harmonic after app_vel criteria",
                ndivide_list = [-1, 1, -2, 2, -3, 3, -4, 4], 
                PSO_nparticle = 2, PSO_maxiter = 2, 
                finer_ndivide= [-2, 2, -3, 3, -4, 4, -5, 5],
                nthread = 6)

dividing layers to 1
Runing with initial smoothing and damping factors. 
 smooth factor is :1.0 damp factor is : 0.5

dividing layers to 1
Runing with initial smoothing and damping factors. 
 smooth factor is :1.0 damp factor is : 0.5
dividing layers to 1
Runing with initial smoothing and damping factors. 
 smooth factor is :1.0 damp factor is : 0.5===============================================
dividing layers to 1
Runing with initial smoothing and damping factors. 
 smooth factor is :1.0 damp factor is : 0.5
dividing layers to 1
Runing with initial smoothing and damping factors. 
 smooth factor is :1.0 damp factor is : 0.5===============================================
dividing layers to 1
Runing with initial smoothing and damping factors. 
 smooth factor is :1.0 damp factor is : 0.5

dividing layers to 1
Runing with increased smoothing and damping factors. 
 smooth factor is :1.2 damp factor is : 0.55
dividing layers to 1
Runing with increased smoothing and damping factors. 
 smooth

<div class="alert alert-block alert-info">
<b>Tip:</b> Again for simplicity i used 6 model in 6 thread.
</div>

The inversion output figures saved.
These figures include the pseudo-initial model from PSO which is the model that generates the lowest difference between
calculated and observed RFs and apparent velocities(the figure with subplots equal to the number of elements in finer_ndivide_list),  Output of mean shear velocity of interpolated PSO pseudo-initial models, and output of shear velocity models of PSO.

<div class="alert alert-block alert-info">
<b>Tip:</b> We can save this station by:
</div>

In [1]:
jrfapp_stobj_file_name = jrfapp_stobj.save_file(file_name='real_data_cad_pso_af_inv.bin')

NameError: name 'jrfapp_stobj' is not defined

#### The inversion output of jrfapp_stobj is saved as a dictionary in jrfapp_stobj.inv_info. 
#### The inv_info dictionary contains several keys. These keys are:

1. all_iter:
> This is a list containing inversion information of the pseudo-initial model either from PSO or grid_search for all divisions according to
> to the finde_ndivide_list. Each inversion information for each iteration is a list that contains several parameters.
> for example jrfapp_stobj.inv_info['all_iter'][0] represents the inversion of the pseudo-initial model with layers divided into 2.
> This list contains these parameters:
> first argument (jrfapp_stobj.inv_info['all_iter'][0][0]) is a list that represents the layer thickness of the model before inversion,
> second argument (jrfapp_stobj.inv_info['all_iter'][0][1]) is a list that represents shear velocities before inversion.
> third argument (jrfapp_stobj.inv_info['all_iter'][0][2]) is an array that represents shear velocity after inversion (estimated).
> forth argument (jrfapp_stobj.inv_info['all_iter'][0][3]) is the norm of the difference between calculated and observed RFs after inversion.
> fifth argument (jrfapp_stobj.inv_info['all_iter'][0][4]) is the norm of the difference between calculated and observed apparent velocity after inversion.
> sixth argument (jrfapp_stobj.inv_info['all_iter'][0][5]) is calculated RFR.
> seventh argument (jrfapp_stobj.inv_info['all_iter'][0][6]) is the calculated apparent velocity.
> eighth argument (jrfapp_stobj.inv_info['all_iter'][0][7]) is the layer thickness after inversion (this is the layer thickness that you
> should use for plotting shear velocities).

2. best_inv:
> This is a list that contains the best model (according to the lowest norm of the difference between calculated and observed RFs and apparent
> velocities) from inversions with the different layering of the pseudo-initial model (best of all_iter).
> The structures of this list are similar to those explained above(initial_layer_thickness, initial_shear_velocity,
> estimated_shear_velocity, RF_norm, apparent_vel_norm, calculated_RFR, calculated_apparent_vel, estimated_layer_thickness).
> This is a key that you want to show in your outputs.

3. mean_vel_inv:
> similar to 2, but for the mean velocity of grid_search (in case you used grid_search as an inversion method).

4. name_pso:
> path to the output of the PSO algorithm in case you want to use force_load in inversion, you can use this filename. only for PSO
>  inversion method.

5. vel_param:
> Object of the velocity parameterization for the pseudo-initial model.
   

<div class="alert alert-block alert-info">
<b>Tip:</b> All the tutorials py files can be found a
</div>